In [2]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
# from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import imageio
import glob
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [134]:
x1 = []
for filename in glob.glob('~/Documents/astroSprint/output_rgb_format_data/lsst_triplets/single_fake2/*.png'):
    im = imageio.imread(filename)
    x1.append(im)

In [135]:
x2 = []
for filename in glob.glob('~/Documents/astroSprint//output_rgb_format_data/lsst_triplets/two_fakes2/*.png'):
    im = imageio.imread(filename)
    x2.append(im)

In [5]:
y1=[]
for i in enumerate(x1):
    y1.append(0)

In [6]:
y2=[]
for i in enumerate(x2):
    y2.append(1)

In [7]:
x = [] 
x.extend(x1)
x.extend(x2)
# x = np.expand_dims(x, axis=1)
x = np.array(x)

In [80]:
y = [] 
y.extend(y1)
y.extend(y2)
# y = to_categorical(y, num_classes=2)
# y = np.array(y)

#### Approach 1 - Training our own VGG16 model from scratch

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.1)

In [10]:
X_train.shape

(412, 224, 224, 3)

In [18]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

In [19]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, 
              metrics=['accuracy', 
                       tf.keras.metrics.AUC(num_thresholds=1200, curve='ROC', summation_method='interpolation')])

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 56, 56, 256)      

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto')
# hist = model.fit_generator(steps_per_epoch=10,generator=(X_train, y_train), validation_data= (X_test, y_test), validation_steps=10,epochs=50,callbacks=[checkpoint,early])
hist = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32,callbacks=[checkpoint,early])

Epoch 1/50
 4/13 [========>.....................] - ETA: 55s - loss: 4722.0685 - accuracy: 0.4095 - auc_2: 0.4117 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["acc"])
plt.plot(hist.history['val_acc'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.savefig("trainvalid.png")

#### Approach 2 - Using weights of pre-trained VGG16 model for classification

In [54]:
from keras.models import Model
from keras.applications import vgg16
vgg_model = vgg16.VGG16(weights='imagenet')
feat_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)
feat_1 =tf.keras.applications.VGG16()

In [55]:
feat_1.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [56]:
# from keras.applications.imagenet_utils import preprocess_input

# img_arr=np.vstack(x)
# processed_imgs= preprocess_input(img_arr.copy())
img_features=feat_extractor.predict(x)
img_features.shape

(458, 4096)

In [81]:
X_train2, X_test2, y_train, y_test = train_test_split(img_features, y, random_state=42, test_size=0.1)

In [110]:
penalty = ['l2','l1']
solver = ['liblinear']
C = [1e-4,1e-3,1e-2,1e-1]
class_weight = ['balanced']
multi_class = ['ovr']

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty, class_weight=class_weight, solver=solver, multi_class=multi_class)

In [111]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_predict
# model = LogisticRegression()
logreg = GridSearchCV(LogisticRegression(), hyperparameters, scoring='roc_auc', cv=5)
logreg.fit(X_train2, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1],
                         'class_weight': ['balanced'], 'multi_class': ['ovr'],
                         'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
             scoring='roc_auc')

In [112]:
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report

print(logreg.best_params_)
y_pred = cross_val_predict(logreg, X_test2, y_test, cv=5)
print('Best Penalty:', logreg.best_estimator_.get_params()['penalty'])
print('Best C:', logreg.best_estimator_.get_params()['C'])
print('Overall AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy of logistic regression classifier on validate set (RFE): {:.2f}'.format(
    logreg.score(X_test2, y_test)))

confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
# Compute precision, recall, F-measure and support
print(classification_report(y_test, y_pred))
specificity = confusion_matrix[0, 0] / (confusion_matrix[0, 0] + confusion_matrix[0, 1])
print('specificity : ', specificity)
sensitivity = confusion_matrix[1, 1] / (confusion_matrix[1, 0] + confusion_matrix[1, 1])
print('sensitivity : ', sensitivity)
print('fawadMetric : ', sensitivity * specificity)

{'C': 0.1, 'class_weight': 'balanced', 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'}
Best Penalty: l1
Best C: 0.1
Overall AUC: 0.47023809523809523
Accuracy of logistic regression classifier on validate set (RFE): 0.48
[[17 11]
 [12  6]]
              precision    recall  f1-score   support

           0       0.59      0.61      0.60        28
           1       0.35      0.33      0.34        18

    accuracy                           0.50        46
   macro avg       0.47      0.47      0.47        46
weighted avg       0.49      0.50      0.50        46

specificity :  0.6071428571428571
sensitivity :  0.3333333333333333
fawadMetric :  0.20238095238095236


In [130]:
n_estimators = [5, 10, 15]
# max_features = [1]
class_weight = ['balanced_subsample']
criterion = ['gini']
min_samples_split = [2, 5]
min_samples_leaf = [2, 5, 10]
# max_depth = [None]
# max_leaf_nodes = [3, 7, None]
# min_impurity_decrease = [0.0]
# min_weight_fraction_leaf = [0]
n_jobs = [-1]
# oob_score = [True]
random_state = [888888]
param_grid = dict(n_estimators=n_estimators, criterion=criterion, min_samples_split=min_samples_split,
                  n_jobs=n_jobs, random_state=random_state, class_weight=class_weight,
                  min_samples_leaf=min_samples_leaf)

In [131]:
X_train2.shape[1]

4096

In [132]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_predict
trained_model = RandomForestClassifier()
rfc = GridSearchCV(estimator=trained_model, param_grid=dict(param_grid, max_features=[X_train2.shape[1]]), scoring='roc_auc', cv=5)
rfc.fit(X_train2, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'class_weight': ['balanced_subsample'],
                         'criterion': ['gini'], 'max_features': [4096],
                         'min_samples_leaf': [2, 5, 10],
                         'min_samples_split': [2, 5],
                         'n_estimators': [5, 10, 15], 'n_jobs': [-1],
                         'random_state': [888888]},
             scoring='roc_auc')

In [133]:
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report

print(rfc.best_params_)
y_pred = cross_val_predict(rfc, X_test2, y_test, cv=5)
print('Overall AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy of Random Forest classifier on validate set (RFE): {:.2f}'.format(
    rfc.score(X_test2, y_test)))

confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
# Compute precision, recall, F-measure and support
print(classification_report(y_test, y_pred))
specificity = confusion_matrix[0, 0] / (confusion_matrix[0, 0] + confusion_matrix[0, 1])
print('specificity : ', specificity)
sensitivity = confusion_matrix[1, 1] / (confusion_matrix[1, 0] + confusion_matrix[1, 1])
print('sensitivity : ', sensitivity)
print('fawadMetric : ', sensitivity * specificity)

{'class_weight': 'balanced_subsample', 'criterion': 'gini', 'max_features': 4096, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 5, 'n_jobs': -1, 'random_state': 888888}
Overall AUC: 0.46230158730158727
Accuracy of Random Forest classifier on validate set (RFE): 0.31
[[15 13]
 [11  7]]
              precision    recall  f1-score   support

           0       0.58      0.54      0.56        28
           1       0.35      0.39      0.37        18

    accuracy                           0.48        46
   macro avg       0.46      0.46      0.46        46
weighted avg       0.49      0.48      0.48        46

specificity :  0.5357142857142857
sensitivity :  0.3888888888888889
fawadMetric :  0.20833333333333334
